In [1]:
from pathlib import Path

from dask.distributed import Client

from excited_workflow.train_models import carbon_tracker


client = Client()

In [2]:
ds_cb = Path("/data/volume_2/EXCITED_prepped_data/CT2022.flux1x1-monthly.nc")
ds_regions = Path("/data/volume_2/EXCITED_prepped_data/regions.nc")
output_dir = Path("/home/cdonnelly")

desired_data = [
    "biomass",
    "spei",
    "modis",
    "era5_monthly",
    "era5_land_monthly",
    "copernicus_landcover"
]

x_keys = ["d2m", "mslhf", "msshf", "ssr", "str", "t2m", "spei", "NIRv", "skt",
            "stl1", "swvl1", "lccs_class"]
y_key = "bio_flux_opt"

In [3]:
ds_input = carbon_tracker.merge_datasets(desired_data, ds_cb)
ds_input

<xarray.Dataset>
Dimensions:     (time: 385, longitude: 360, latitude: 180)
Coordinates:
  * time        (time) datetime64[ns] 1990-01-01 1990-02-01 ... 2022-01-01
  * longitude   (longitude) float64 -179.5 -178.5 -177.5 ... 177.5 178.5 179.5
  * latitude    (latitude) float64 -89.5 -88.5 -87.5 -86.5 ... 87.5 88.5 89.5
Data variables: (12/24)
    biomass     (time, latitude, longitude) float64 dask.array<chunksize=(385, 180, 360), meta=np.ndarray>
    spei        (time, latitude, longitude) float32 dask.array<chunksize=(96, 180, 360), meta=np.ndarray>
    NDVI        (time, latitude, longitude) float32 dask.array<chunksize=(385, 180, 360), meta=np.ndarray>
    NIRv        (time, latitude, longitude) float32 dask.array<chunksize=(385, 180, 360), meta=np.ndarray>
    d2m         (time, latitude, longitude) float32 dask.array<chunksize=(132, 180, 360), meta=np.ndarray>
    mslhf       (time, latitude, longitude) float32 dask.array<chunksize=(132, 180, 360), meta=np.ndarray>
    ...          ...
    stl4        (time, latitude, longitude) float32 dask.array<chunksize=(132, 180, 360), meta=np.ndarray>
    swvl1       (time, latitude, longitude) float32 dask.array<chunksize=(132, 180, 360), meta=np.ndarray>
    swvl2       (time, latitude, longitude) float32 dask.array<chunksize=(132, 180, 360), meta=np.ndarray>
    swvl3       (time, latitude, longitude) float32 dask.array<chunksize=(132, 180, 360), meta=np.ndarray>
    swvl4       (time, latitude, longitude) float32 dask.array<chunksize=(132, 180, 360), meta=np.ndarray>
    lccs_class  (time, latitude, longitude) float32 dask.array<chunksize=(385, 180, 360), meta=np.ndarray>
Attributes:
    title:        Changes in Global Terrestrial Live Biomass over the 21st Ce...
    version:      2
    institution:  Jet Propulsion Laboratory, California Institute of Technology
    source:       Estimates of carbon stock changes of live woody biomass fro...
    history:      \n2022-08-18: downloaded source from https://zenodo.org/rec...
    references:   \n@Xu2021{\n  author = {Xu, Liang, Saatchi, Sassan S., Yang...

In [4]:
ds_na = carbon_tracker.mask_region(ds_regions, ds_cb, ds_input)
ds_na

<xarray.Dataset>
Dimensions:           (time: 240, latitude: 45, longitude: 85)
Coordinates:
  * longitude         (longitude) float64 -139.5 -138.5 -137.5 ... -56.5 -55.5
  * latitude          (latitude) float64 15.5 16.5 17.5 18.5 ... 57.5 58.5 59.5
  * time              (time) datetime64[ns] 2000-01-01 2000-02-01 ... 2019-12-01
Data variables: (12/26)
    bio_flux_opt      (time, latitude, longitude) float64 nan nan ... nan nan
    transcom_regions  (latitude, longitude) float64 nan nan nan ... nan nan nan
    biomass           (time, latitude, longitude) float64 nan nan ... nan nan
    spei              (time, latitude, longitude) float64 nan nan ... nan nan
    NDVI              (time, latitude, longitude) float64 nan nan ... nan nan
    NIRv              (time, latitude, longitude) float64 nan nan ... nan nan
    ...                ...
    stl4              (time, latitude, longitude) float64 nan nan ... nan nan
    swvl1             (time, latitude, longitude) float64 nan nan ... nan nan
    swvl2             (time, latitude, longitude) float64 nan nan ... nan nan
    swvl3             (time, latitude, longitude) float64 nan nan ... nan nan
    swvl4             (time, latitude, longitude) float64 nan nan ... nan nan
    lccs_class        (time, latitude, longitude) float32 nan nan ... nan nan
Attributes:
    averaging_period_length_hours:  744
    email:                          carbontracker.team@noaa.gov
    url:                            http://carbontracker.noaa.gov
    institution:                    NOAA Earth System Research Laboratory
    Conventions:                    CF-1.5
    history:                        Time-stamp: <Orion-login-1.HPC.MsState.Ed...
    NCO:                            netCDF Operators version 4.9.3 (Homepage ...
    version:                        CT2022 1x1 3-hourly fluxes as of 2023-01-...

In [5]:
carbon_tracker.validate_model(ds_na, 5, x_keys, y_key, output_dir)

,Description,Value
0,Session id,8057
1,Target,bio_flux_opt
2,Target type,Regression
3,Original data shape,"(21325, 13)"
4,Transformed data shape,"(21325, 13)"
5,Transformed train set shape,"(14927, 13)"
6,Transformed test set shape,"(6398, 13)"
7,Numeric features,12
8,Preprocess,True
9,Imputation type,simple


,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE,TT (Sec)
lightgbm,Light Gradient Boosting Machine,0.3,0.3,0.5,0.8,0.2,4.3,0.3


,Description,Value
0,Session id,3340
1,Target,bio_flux_opt
2,Target type,Regression
3,Original data shape,"(21325, 13)"
4,Transformed data shape,"(21325, 13)"
5,Transformed train set shape,"(14927, 13)"
6,Transformed test set shape,"(6398, 13)"
7,Numeric features,12
8,Preprocess,True
9,Imputation type,simple


,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE,TT (Sec)
lightgbm,Light Gradient Boosting Machine,0.3,0.3,0.5,0.7,0.2,3.5,0.1


,Description,Value
0,Session id,4543
1,Target,bio_flux_opt
2,Target type,Regression
3,Original data shape,"(21325, 13)"
4,Transformed data shape,"(21325, 13)"
5,Transformed train set shape,"(14927, 13)"
6,Transformed test set shape,"(6398, 13)"
7,Numeric features,12
8,Preprocess,True
9,Imputation type,simple


,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE,TT (Sec)
lightgbm,Light Gradient Boosting Machine,0.3,0.3,0.5,0.7,0.2,4.6,0.1


,Description,Value
0,Session id,533
1,Target,bio_flux_opt
2,Target type,Regression
3,Original data shape,"(21325, 13)"
4,Transformed data shape,"(21325, 13)"
5,Transformed train set shape,"(14927, 13)"
6,Transformed test set shape,"(6398, 13)"
7,Numeric features,12
8,Preprocess,True
9,Imputation type,simple


,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE,TT (Sec)
lightgbm,Light Gradient Boosting Machine,0.3,0.3,0.5,0.7,0.2,3.9,0.1


,Description,Value
0,Session id,2850
1,Target,bio_flux_opt
2,Target type,Regression
3,Original data shape,"(21325, 13)"
4,Transformed data shape,"(21325, 13)"
5,Transformed train set shape,"(14927, 13)"
6,Transformed test set shape,"(6398, 13)"
7,Numeric features,12
8,Preprocess,True
9,Imputation type,simple


,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE,TT (Sec)
lightgbm,Light Gradient Boosting Machine,0.3,0.2,0.5,0.7,0.2,3.8,0.1


In [6]:
onnx_model = carbon_tracker.save_model(ds_na, x_keys, y_key, output_dir)

,Description,Value
0,Session id,1378
1,Target,bio_flux_opt
2,Target type,Regression
3,Original data shape,"(26656, 13)"
4,Transformed data shape,"(26656, 13)"
5,Transformed train set shape,"(18659, 13)"
6,Transformed test set shape,"(7997, 13)"
7,Numeric features,12
8,Preprocess,True
9,Imputation type,simple


,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE,TT (Sec)
lightgbm,Light Gradient Boosting Machine,0.3,0.3,0.5,0.7,0.2,3.7,0.1


The maximum opset needed by this model is only 8.


In [14]:
import pandas as pd
import numpy as np

df = ds_na.to_dataframe().dropna()
df = carbon_tracker.create_df(df, x_keys, y_key)
df

d2m       mslhf      msshf         ssr  \
time       latitude longitude                                                  
2000-08-01 16.5     -97.5      291.160736 -123.552444 -29.256828  17632392.0   
           18.5     -101.5     293.513855  -93.125107 -56.438019  18068404.0   
           19.5     -101.5     285.543335  -89.911835 -44.041389  15922805.0   
           20.5     -100.5     285.119324  -68.271637 -76.867363  18879166.0   
           21.5     -98.5      296.115051 -104.443359 -43.687386  16755104.0   
...                                   ...         ...        ...         ...   
2019-12-01 51.5     -101.5     256.830872   -0.664703  11.963760   2044303.0   
           52.5     -112.5     262.490692   -1.074692  15.552032   1481686.0   
           53.5     -116.5     261.434357   -3.534576  20.831009   1808079.0   
           54.5     -114.5     260.227997   -2.052338  16.436630   1571385.0   
           56.5     -131.5     269.274261   -3.085175   0.307220    365373.0   

                                     str         t2m      spei      NIRv  \
time       latitude longitude                                              
2000-08-01 16.5     -97.5     -4539167.0  294.455078  0.402637  0.264352   
           18.5     -101.5    -5339288.5  299.283936 -0.300553  0.281680   
           19.5     -101.5    -4355907.5  288.824677 -0.292989  0.243739   
           20.5     -100.5    -6372623.5  292.068542  0.086668  0.152103   
           21.5     -98.5     -4156947.5  300.844299  0.120198  0.252774   
...                                  ...         ...       ...       ...   
2019-12-01 51.5     -101.5    -3697116.5  258.765778  1.172457 -0.031652   
           52.5     -112.5    -3616690.5  264.937561  1.633417 -0.039948   
           53.5     -116.5    -3773900.0  264.205383  2.066818  0.017840   
           54.5     -114.5    -3568210.5  262.839630  0.981084 -0.007025   
           56.5     -131.5    -1425005.5  270.888672 -0.181532 -0.038799   

                                      skt        stl1     swvl1  lccs_class  \
time       latitude longitude                                                 
2000-08-01 16.5     -97.5      293.800140  294.488708  0.405656        90.0   
           18.5     -101.5     299.144745  299.822601  0.334316        60.0   
           19.5     -101.5     287.316406  287.972260  0.406064        90.0   
           20.5     -100.5     293.144592  293.650024  0.195828        11.0   
           21.5     -98.5      300.711945  301.543549  0.287784        50.0   
...                                   ...         ...       ...         ...   
2019-12-01 51.5     -101.5     258.316467  270.511261  0.444990        11.0   
           52.5     -112.5     263.270752  268.994141  0.311979        11.0   
           53.5     -116.5     261.317444  272.363464  0.482138        71.0   
           54.5     -114.5     261.786438  269.405945  0.462469        11.0   
           56.5     -131.5     265.012054  270.157806  0.281438        71.0   

                               bio_flux_opt  
time       latitude longitude                
2000-08-01 16.5     -97.5         -0.601002  
           18.5     -101.5        -2.348510  
           19.5     -101.5        -0.825588  
           20.5     -100.5        -1.438290  
           21.5     -98.5         -0.263195  
...                                     ...  
2019-12-01 51.5     -101.5         0.184300  
           52.5     -112.5         0.023272  
           53.5     -116.5         0.954168  
           54.5     -114.5         0.848743  
           56.5     -131.5         0.552836  

[26656 rows x 13 columns]

In [15]:
from onnxruntime import InferenceSession

with open(output_dir / "lightgbm.onnx", "rb") as f:
    model = f.read()

sess = InferenceSession(model)
predictions_onnx = sess.run(None, {'X': df.to_numpy()})[0]

InvalidArgument: [ONNXRuntimeError] : 2 : INVALID_ARGUMENT : Unexpected input data type. Actual: (tensor(double)) , expected: (tensor(float))